# Realtime blockchain monitoring

In [14]:
!PROJECT_ID=$(gcloud config list --format 'value(core.project)')

%run eth_monitoring.py \
    --subscription projects/forrest-test-project-333203/subscriptions/ethTransactionParser \
    --output_topic projects/forrest-test-project-333203/topics/blockchain.eth.whale_alert \
    --sql "SELECT hash, from_address, to_address, value / power(10,18) as value from PCOLLECTION \
           WHERE value >= 100 * power(10,18) \
           AND receipt_status = 1" \
    --region us-central1 \
    --runner DataflowRunner \
    --project forrest-test-project-333203 \
    --temp_location gs://forrest-bigdata-bucket/dataflow/temp \
    --staging_location gs://forrest-bigdata-bucket/dataflow/staging \
    --job_name eth-whale-alert \
    --service_account_email "notebook@forrest-test-project-333203.iam.gserviceaccount.com" \
    --enable_streaming_engine true \
    --network bigdata-network \
    --subnetwork regions/us-central1/subnetworks/dataflow-network \
    --use_public_ips false


INFO:apache_beam.utils.subprocess_server:Using cached job server jar from https://repo.maven.apache.org/maven2/org/apache/beam/beam-sdks-java-extensions-sql-expansion-service/2.33.0/beam-sdks-java-extensions-sql-expansion-service-2.33.0.jar
INFO:apache_beam.utils.subprocess_server:Starting service with ['java' '-jar' '/root/.apache_beam/cache/jars/beam-sdks-java-extensions-sql-expansion-service-2.33.0.jar' '58489']
INFO:apache_beam.utils.subprocess_server:b'Starting expansion service at localhost:58489'
INFO:apache_beam.utils.subprocess_server:b'Dec 21, 2021 10:00:03 AM org.apache.beam.sdk.expansion.service.ExpansionService loadRegisteredTransforms'
INFO:apache_beam.utils.subprocess_server:b'INFO: Registering external transforms: [beam:external:java:sql:v1, beam:external:java:generate_sequence:v1]'
INFO:apache_beam.utils.subprocess_server:b'\tbeam:external:java:sql:v1: org.apache.beam.sdk.expansion.service.ExpansionService$ExternalTransformRegistrarLoader$$Lambda$47/93314457@6591f517'


RuntimeError: Service failed to start up with error 127

In [6]:
import apache_beam

## whale alert

In [8]:
DECIMAL = 10 ** 18
WHALE_THRESHOLD = 100

p = beam.Pipeline(options=options)
whale_alert = (
    p
    | "read from pubsub" >> beam.io.ReadFromPubSub(subscription=subscription)
    | 'transform to json' >> beam.Map(json.loads)
    | 'create beam row' >> beam.Map(
        lambda x: beam.Row(
            hash=elem["hash"],
            transaction_index=elem["transaction_index"],
            from_address=elem["from_address"],
            to_address=elem["from_address"],
            value=elem["value"] / DECIMAL,
            receipt_status=elem["receipt_status"],
            block_timestamp=elem["block_timestamp"],
            block_hash=elem["block_hash"]))
    | 'filter whale transaction' >> SqlTransform(
        f"""
          select * from PCOLLECTION
          where value >= {WHALE_THRESHOLD}
          and receipt_status = 1
        """)
    | 'transform to dict' >> beam.Map(
        lambda row: {
            "blockchain": "eth",
            "hash": row.hash,
            "transaction_index": row.transaction_index,
            "from_address": row.from_address,
            "to_address": row.from_address,
            "value": row.value,
            "receipt_status": row.receipt_status,
            "block_timestamp": row.block_timestamp,
            "block_hash": row.block_hash})
    | 'transform to string' >> beam.Map(json.dumps)
    | 'UTF-8 encode' >> beam.Map(lambda s: s.encode("utf-8"))
    # | 'write to pubsub' >> beam.io.WriteToPubSub(topic=output_topic)
)

ERROR:apache_beam.utils.subprocess_server:Starting job service with ['java', '-jar', '/root/.apache_beam/cache/jars/beam-sdks-java-extensions-sql-expansion-service-2.33.0.jar', '59177']
ERROR:apache_beam.utils.subprocess_server:Error bringing up service
Traceback (most recent call last):
  File "/root/apache-beam-2.33.0/lib/python3.7/site-packages/apache_beam/utils/subprocess_server.py", line 86, in start
    'Service failed to start up with error %s' % self._process.poll())
RuntimeError: Service failed to start up with error 127


RuntimeError: Service failed to start up with error 127

In [ ]:
from apache_beam.runners import DataflowRunner
runner = DataflowRunner()
runner.run_pipeline(p, options=options)

In [3]:
!java -jar /root/.apache_beam/cache/jars/beam-sdks-java-extensions-sql-expansion-service-2.33.0.jar 49619

Starting expansion service at localhost:49619
Dec 20, 2021 12:05:30 PM org.apache.beam.sdk.expansion.service.ExpansionService loadRegisteredTransforms
INFO: Registering external transforms: [beam:external:java:sql:v1, beam:external:java:generate_sequence:v1]
	beam:external:java:sql:v1: org.apache.beam.sdk.expansion.service.ExpansionService$ExternalTransformRegistrarLoader$$Lambda$47/93314457@6591f517
	beam:external:java:generate_sequence:v1: org.apache.beam.sdk.expansion.service.ExpansionService$ExternalTransformRegistrarLoader$$Lambda$47/93314457@345965f2
java: relocation error: /usr/lib/jvm/java-8-openjdk-amd64/jre/lib/amd64/libnio.so: symbol initInetAddressIDs version SUNWprivate_1.1 not defined in file libnet.so with link time reference
